In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence

import os
import random

In [8]:
batch_size=128
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128
result_path="results_np_z_y_hat_parmu/"

In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [5]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
    
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400, 400)
        self.g_3 = nn.Linear(400, 400)
        self.g_4 = nn.Linear(400, 400)
        self.g_y_mu = nn.Linear(400, 1)
        self.g_y_sigma = nn.Linear(400, 1)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self,z_sample, x_target):
        z_et_x = torch.cat([z_sample, x_target], dim=2)
        input = F.relu(self.g_1(z_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        input = F.relu(self.g_4(input))
        y_mu=self.g_y_mu(input)
        y_sigma=0.1 + 0.9 * F.softplus(self.g_y_sigma(input))
        return y_mu,y_sigma
    
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        mu = self.r_to_z_mean(r)
        logvar = self.r_to_z_logvar(r)
        sigma=0.1+0.9*torch.sigmoid(logvar)
        return mu, sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        
        #produire z
        z_context_mu,z_context_sigma = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        q_context = Normal(z_context_mu, z_context_sigma)
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_target_mu,z_target_sigma = self.xy_to_z_params(x_context, y_context)
            q_target = Normal(z_target_mu, z_target_sigma) 
            z_sample = q_target.rsample()
            
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            
            # Get parameters of output distribution
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)

            return p_y_pred, q_target, q_context
        else:  # at test time we don't have the image so we use only the context
            z_sample = q_context.rsample()
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            # Predict target points based on context
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)
            return p_y_pred,q_context,q_context

In [6]:

def np_loss(p_y_pred, y_target, q_target, q_context):
    
    #return logprob + KLD
    log_likelihood = p_y_pred.log_prob(y_target).mean(dim=0).sum()
    # KL has shape (batch_size, r_dim). Take mean over batch and sum over
    # r_dim (since r_dim is dimension of normal distribution)
    kl = kl_divergence(q_target, q_context).mean(dim=0).sum()
    return -log_likelihood + kl
    
    

In [10]:
model = NP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [14]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        p_y_pred, q_target, q_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(p_y_pred, y_all, q_target, q_context)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [15]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            p_y_pred, q_target, q_context = model(x_context, y_context)
        
            
            test_loss += np_loss(p_y_pred, y_all, q_target, q_context).item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    recons2=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):
                        p_y_pred, _, _ = model(x_context, y_context)
                        
                        recons.append(p_y_pred.rsample()[:num_examples])
                        recons1.append(p_y_pred.mean[:num_examples])  
                        recons2.append(p_y_pred.stddev[:num_examples])  
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons2 = torch.cat(recons2).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1,recons2])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [16]:
for epoch in range(87, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 87 [0/60000 (0%)]	Loss: -3.082180
Train Epoch: 87 [128/60000 (0%)]	Loss: 1.188691
Train Epoch: 87 [256/60000 (0%)]	Loss: -2.935480
Train Epoch: 87 [384/60000 (1%)]	Loss: -2.806319
Train Epoch: 87 [512/60000 (1%)]	Loss: -2.037050
Train Epoch: 87 [640/60000 (1%)]	Loss: -1.637526
Train Epoch: 87 [768/60000 (1%)]	Loss: -1.580763
Train Epoch: 87 [896/60000 (1%)]	Loss: -1.539144
Train Epoch: 87 [1024/60000 (2%)]	Loss: -2.187034
Train Epoch: 87 [1152/60000 (2%)]	Loss: -2.447201
Train Epoch: 87 [1280/60000 (2%)]	Loss: -2.606909
Train Epoch: 87 [1408/60000 (2%)]	Loss: -2.631627
Train Epoch: 87 [1536/60000 (3%)]	Loss: -2.833697
Train Epoch: 87 [1664/60000 (3%)]	Loss: -2.678462
Train Epoch: 87 [1792/60000 (3%)]	Loss: -2.419366
Train Epoch: 87 [1920/60000 (3%)]	Loss: -2.477812
Train Epoch: 87 [2048/60000 (3%)]	Loss: -2.819392
Train Epoch: 87 [2176/60000 (4%)]	Loss: -2.575150
Train Epoch: 87 [2304/60000 (4%)]	Loss: -2.767932
Train Epoch: 87 [2432/60000 (4%)]	Loss: -2.776758
Train Epoch

Train Epoch: 87 [20608/60000 (34%)]	Loss: -2.907620
Train Epoch: 87 [20736/60000 (35%)]	Loss: -3.050639
Train Epoch: 87 [20864/60000 (35%)]	Loss: -3.075966
Train Epoch: 87 [20992/60000 (35%)]	Loss: -3.109695
Train Epoch: 87 [21120/60000 (35%)]	Loss: -3.334300
Train Epoch: 87 [21248/60000 (35%)]	Loss: -3.068986
Train Epoch: 87 [21376/60000 (36%)]	Loss: -2.914736
Train Epoch: 87 [21504/60000 (36%)]	Loss: -2.932645
Train Epoch: 87 [21632/60000 (36%)]	Loss: -2.881518
Train Epoch: 87 [21760/60000 (36%)]	Loss: -3.088144
Train Epoch: 87 [21888/60000 (36%)]	Loss: -2.963249
Train Epoch: 87 [22016/60000 (37%)]	Loss: -3.191315
Train Epoch: 87 [22144/60000 (37%)]	Loss: -3.139002
Train Epoch: 87 [22272/60000 (37%)]	Loss: -2.842208
Train Epoch: 87 [22400/60000 (37%)]	Loss: -3.239524
Train Epoch: 87 [22528/60000 (38%)]	Loss: -2.853788
Train Epoch: 87 [22656/60000 (38%)]	Loss: -2.904053
Train Epoch: 87 [22784/60000 (38%)]	Loss: -3.079805
Train Epoch: 87 [22912/60000 (38%)]	Loss: -3.067222
Train Epoch:

Train Epoch: 87 [40832/60000 (68%)]	Loss: -2.872589
Train Epoch: 87 [40960/60000 (68%)]	Loss: -2.970834
Train Epoch: 87 [41088/60000 (68%)]	Loss: -2.791021
Train Epoch: 87 [41216/60000 (69%)]	Loss: -3.087197
Train Epoch: 87 [41344/60000 (69%)]	Loss: -3.081291
Train Epoch: 87 [41472/60000 (69%)]	Loss: -2.864009
Train Epoch: 87 [41600/60000 (69%)]	Loss: -2.996753
Train Epoch: 87 [41728/60000 (70%)]	Loss: -2.928898
Train Epoch: 87 [41856/60000 (70%)]	Loss: -3.054400
Train Epoch: 87 [41984/60000 (70%)]	Loss: -3.177228
Train Epoch: 87 [42112/60000 (70%)]	Loss: -3.024163
Train Epoch: 87 [42240/60000 (70%)]	Loss: -3.116023
Train Epoch: 87 [42368/60000 (71%)]	Loss: -2.190905
Train Epoch: 87 [42496/60000 (71%)]	Loss: -2.876881
Train Epoch: 87 [42624/60000 (71%)]	Loss: -2.765496
Train Epoch: 87 [42752/60000 (71%)]	Loss: -2.932101
Train Epoch: 87 [42880/60000 (71%)]	Loss: -2.963105
Train Epoch: 87 [43008/60000 (72%)]	Loss: -3.131891
Train Epoch: 87 [43136/60000 (72%)]	Loss: -2.996643
Train Epoch:

Train Epoch: 88 [896/60000 (1%)]	Loss: -3.210895
Train Epoch: 88 [1024/60000 (2%)]	Loss: -2.355079
Train Epoch: 88 [1152/60000 (2%)]	Loss: -2.971742
Train Epoch: 88 [1280/60000 (2%)]	Loss: -2.977236
Train Epoch: 88 [1408/60000 (2%)]	Loss: -3.172798
Train Epoch: 88 [1536/60000 (3%)]	Loss: -3.170920
Train Epoch: 88 [1664/60000 (3%)]	Loss: -3.210899
Train Epoch: 88 [1792/60000 (3%)]	Loss: -3.132704
Train Epoch: 88 [1920/60000 (3%)]	Loss: -3.188905
Train Epoch: 88 [2048/60000 (3%)]	Loss: -2.837752
Train Epoch: 88 [2176/60000 (4%)]	Loss: -3.421345
Train Epoch: 88 [2304/60000 (4%)]	Loss: -3.014694
Train Epoch: 88 [2432/60000 (4%)]	Loss: -2.816552
Train Epoch: 88 [2560/60000 (4%)]	Loss: -2.669622
Train Epoch: 88 [2688/60000 (4%)]	Loss: -2.794441
Train Epoch: 88 [2816/60000 (5%)]	Loss: -2.811380
Train Epoch: 88 [2944/60000 (5%)]	Loss: -2.994836
Train Epoch: 88 [3072/60000 (5%)]	Loss: -2.551404
Train Epoch: 88 [3200/60000 (5%)]	Loss: -2.909654
Train Epoch: 88 [3328/60000 (6%)]	Loss: -2.879936
T

Train Epoch: 88 [21376/60000 (36%)]	Loss: -3.171627
Train Epoch: 88 [21504/60000 (36%)]	Loss: -3.037426
Train Epoch: 88 [21632/60000 (36%)]	Loss: -2.937056
Train Epoch: 88 [21760/60000 (36%)]	Loss: -3.079432
Train Epoch: 88 [21888/60000 (36%)]	Loss: -3.171532
Train Epoch: 88 [22016/60000 (37%)]	Loss: -3.009366
Train Epoch: 88 [22144/60000 (37%)]	Loss: -3.033118
Train Epoch: 88 [22272/60000 (37%)]	Loss: -3.213776
Train Epoch: 88 [22400/60000 (37%)]	Loss: -3.040295
Train Epoch: 88 [22528/60000 (38%)]	Loss: -3.161528
Train Epoch: 88 [22656/60000 (38%)]	Loss: -3.197536
Train Epoch: 88 [22784/60000 (38%)]	Loss: -3.019566
Train Epoch: 88 [22912/60000 (38%)]	Loss: -2.796914
Train Epoch: 88 [23040/60000 (38%)]	Loss: -2.961504
Train Epoch: 88 [23168/60000 (39%)]	Loss: -3.252102
Train Epoch: 88 [23296/60000 (39%)]	Loss: -3.164525
Train Epoch: 88 [23424/60000 (39%)]	Loss: -3.118039
Train Epoch: 88 [23552/60000 (39%)]	Loss: -3.273531
Train Epoch: 88 [23680/60000 (39%)]	Loss: -3.097673
Train Epoch:

...

Train Epoch: 132 [59392/60000 (99%)]	Loss: -4.605675
Train Epoch: 132 [59520/60000 (99%)]	Loss: -4.893700
Train Epoch: 132 [59648/60000 (99%)]	Loss: -5.271584
Train Epoch: 132 [59776/60000 (100%)]	Loss: -5.149182
Train Epoch: 132 [44928/60000 (100%)]	Loss: -4.461948
====> Epoch: 132 Average loss: -4.7658
====> Test set loss: -5.0177
Train Epoch: 133 [0/60000 (0%)]	Loss: -3.324229
Train Epoch: 133 [128/60000 (0%)]	Loss: -4.724474
Train Epoch: 133 [256/60000 (0%)]	Loss: -5.020652
Train Epoch: 133 [384/60000 (1%)]	Loss: -5.080657
Train Epoch: 133 [512/60000 (1%)]	Loss: -4.770774
Train Epoch: 133 [640/60000 (1%)]	Loss: -4.995505
Train Epoch: 133 [768/60000 (1%)]	Loss: -4.957911
Train Epoch: 133 [896/60000 (1%)]	Loss: -4.730538
Train Epoch: 133 [1024/60000 (2%)]	Loss: -5.066970
Train Epoch: 133 [1152/60000 (2%)]	Loss: -4.981986
Train Epoch: 133 [1280/60000 (2%)]	Loss: -4.824857
Train Epoch: 133 [1408/60000 (2%)]	Loss: -4.947284
Train Epoch: 133 [1536/60000 (3%)]	Loss: -4.903241
Train Epoch:

Train Epoch: 133 [19328/60000 (32%)]	Loss: -5.011500
Train Epoch: 133 [19456/60000 (32%)]	Loss: -5.043919
Train Epoch: 133 [19584/60000 (33%)]	Loss: -4.376301
Train Epoch: 133 [19712/60000 (33%)]	Loss: -4.906216
Train Epoch: 133 [19840/60000 (33%)]	Loss: -4.981998
Train Epoch: 133 [19968/60000 (33%)]	Loss: -4.978896
Train Epoch: 133 [20096/60000 (33%)]	Loss: -4.339766
Train Epoch: 133 [20224/60000 (34%)]	Loss: -5.098927
Train Epoch: 133 [20352/60000 (34%)]	Loss: -5.141216
Train Epoch: 133 [20480/60000 (34%)]	Loss: -5.018085
Train Epoch: 133 [20608/60000 (34%)]	Loss: -5.155190
Train Epoch: 133 [20736/60000 (35%)]	Loss: -4.792689
Train Epoch: 133 [20864/60000 (35%)]	Loss: -4.940498
Train Epoch: 133 [20992/60000 (35%)]	Loss: -2.631121
Train Epoch: 133 [21120/60000 (35%)]	Loss: -4.741528
Train Epoch: 133 [21248/60000 (35%)]	Loss: -4.922884
Train Epoch: 133 [21376/60000 (36%)]	Loss: -3.351195
Train Epoch: 133 [21504/60000 (36%)]	Loss: -5.088899
Train Epoch: 133 [21632/60000 (36%)]	Loss: -4.

Train Epoch: 133 [39168/60000 (65%)]	Loss: -5.074210
Train Epoch: 133 [39296/60000 (65%)]	Loss: -4.992973
Train Epoch: 133 [39424/60000 (66%)]	Loss: -5.133026
Train Epoch: 133 [39552/60000 (66%)]	Loss: -5.245290
Train Epoch: 133 [39680/60000 (66%)]	Loss: -5.048337
Train Epoch: 133 [39808/60000 (66%)]	Loss: -3.222149
Train Epoch: 133 [39936/60000 (67%)]	Loss: -4.944650
Train Epoch: 133 [40064/60000 (67%)]	Loss: -4.896585
Train Epoch: 133 [40192/60000 (67%)]	Loss: -4.909229
Train Epoch: 133 [40320/60000 (67%)]	Loss: -4.769423
Train Epoch: 133 [40448/60000 (67%)]	Loss: -4.992514
Train Epoch: 133 [40576/60000 (68%)]	Loss: -4.916846
Train Epoch: 133 [40704/60000 (68%)]	Loss: -4.952332
Train Epoch: 133 [40832/60000 (68%)]	Loss: -5.252286
Train Epoch: 133 [40960/60000 (68%)]	Loss: -5.035651
Train Epoch: 133 [41088/60000 (68%)]	Loss: -4.982973
Train Epoch: 133 [41216/60000 (69%)]	Loss: -4.831000
Train Epoch: 133 [41344/60000 (69%)]	Loss: -4.308528
Train Epoch: 133 [41472/60000 (69%)]	Loss: -5.

Train Epoch: 133 [59008/60000 (98%)]	Loss: -4.825149
Train Epoch: 133 [59136/60000 (99%)]	Loss: -5.028503
Train Epoch: 133 [59264/60000 (99%)]	Loss: -4.515643
Train Epoch: 133 [59392/60000 (99%)]	Loss: -4.972460
Train Epoch: 133 [59520/60000 (99%)]	Loss: -5.172498
Train Epoch: 133 [59648/60000 (99%)]	Loss: -1.747522
Train Epoch: 133 [59776/60000 (100%)]	Loss: -4.880909
Train Epoch: 133 [44928/60000 (100%)]	Loss: -6.735936
====> Epoch: 133 Average loss: -4.8197
====> Test set loss: -4.9218
Train Epoch: 134 [0/60000 (0%)]	Loss: -4.913137
Train Epoch: 134 [128/60000 (0%)]	Loss: -4.474683
Train Epoch: 134 [256/60000 (0%)]	Loss: -5.277809
Train Epoch: 134 [384/60000 (1%)]	Loss: -4.809884
Train Epoch: 134 [512/60000 (1%)]	Loss: -5.083010
Train Epoch: 134 [640/60000 (1%)]	Loss: -4.994432
Train Epoch: 134 [768/60000 (1%)]	Loss: -5.135287
Train Epoch: 134 [896/60000 (1%)]	Loss: -5.029896
Train Epoch: 134 [1024/60000 (2%)]	Loss: -5.091570
Train Epoch: 134 [1152/60000 (2%)]	Loss: -5.112516
Train 

KeyboardInterrupt: 

In [17]:
torch.save(model.state_dict(), result_path+"model_{}.pt".format(r_dim))

In [70]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs=500

In [13]:

model.load_state_dict(torch.load(result_path+"model.pt"))
model.eval()

NP(
  (h_1): Linear(in_features=3, out_features=400, bias=True)
  (h_2): Linear(in_features=400, out_features=400, bias=True)
  (h_3): Linear(in_features=400, out_features=128, bias=True)
  (r_to_z_mean): Linear(in_features=128, out_features=128, bias=True)
  (r_to_z_logvar): Linear(in_features=128, out_features=128, bias=True)
  (g_1): Linear(in_features=130, out_features=400, bias=True)
  (g_2): Linear(in_features=400, out_features=400, bias=True)
  (g_3): Linear(in_features=400, out_features=400, bias=True)
  (g_4): Linear(in_features=400, out_features=400, bias=True)
  (g_y_mu): Linear(in_features=400, out_features=1, bias=True)
  (g_y_sigma): Linear(in_features=400, out_features=1, bias=True)
)